In [30]:
import pandas as pd
import numpy as np
import time
import fpGrowth 

In [2]:
#预处理数据
def pre_data(df):
    df['timestamp'] = pd.to_datetime(df['sldat'])
    
    #分别按vipno、sldat排序
    data = df.sort_values(by=['vipno', 'sldat'])
    
    #分组
    data['rank'] = data['timestamp'].groupby(data['vipno']).rank(ascending=0, method='first')
    
    #取每个vipno的前60%的数据
    data = data.groupby(['vipno'], as_index = False).apply(lambda x: x[x['rank'] <=  round(0.6 * x['rank'].max())])
    #整理
    data = data.drop(['timestamp', 'rank'], axis = 1).reset_index(drop=True)
    return data

In [3]:
#提出pluno的数据
def pre_pluno(data):
    #丢弃多余的列
    data_pluno = data.drop(['sldat', 'vipno', 'dptno', 'bndno'], axis = 1)
    
    #合并订单
    data_pluno['value'] = data_pluno['pluno']
    data_pluno = data_pluno.pivot_table(data_pluno , index = ['uid'], columns = 'pluno')
    
    #整理
    data_pluno = data_pluno.fillna(0).transpose().loc['value'].transpose()
    del data_pluno.index.name
    del data_pluno.transpose().index.name
    
    #将dataframe转为array
    array_pluno = []
    for row in data_pluno.as_matrix():
        array_pluno.append([x for x in row if x != 0.0])
    return array_pluno

In [28]:
def fp_pluno(data):
    thresholds = [2, 4, 8, 16, 32, 64]
    array_pluno = pre_pluno(data)
    for n in thresholds:
        freq_sets = fpGrowth.fpGrowth(array_pluno, n)
        print("\n for pluno, threshold: ", n)
        for k in freq_sets:
            if len(k) >= 2:
                print (k, freq_sets[k])

In [31]:
if __name__ == "__main__":
    old_df = pd.read_csv('../trade.csv', usecols=['uid', 'vipno', 'sldat', 'pluno', 'dptno', 'bndno'])
    #旧数据
    old_data = pre_data(old_df)
    start = time.clock()
    fp_pluno(old_data)
    time_old = time.clock() - start
    #新数据
    new_df = pd.read_csv('../trade_new.csv', usecols=['uid', 'vipno', 'sldatime', 'pluno', 'dptno', 'bndno'])
    new_df.rename(columns={ new_df.columns[1]: "sldat" }, inplace=True)    
    new_data = pre_data(new_df)
    start = time.clock()
    fp_pluno(new_data)
    time_new = time.clock() - start
    
    print("fp_pluno,for old data: ", time_old)
    print("fp_pluno,for new data: ", time_new)


 for pluno, threshold:  2
frozenset({11501090.0, 30380002.0}) 2
frozenset({25111048.0, 10150033.0}) 2
frozenset({10150033.0, 27200170.0}) 2
frozenset({25111048.0, 10150033.0, 27200170.0}) 2
frozenset({10150033.0, 30380002.0}) 2
frozenset({25111048.0, 14905010.0}) 2
frozenset({10150032.0, 30380002.0}) 2
frozenset({10150032.0, 10000011.0}) 2
frozenset({10150032.0, 30380002.0, 10000011.0}) 2
frozenset({10150032.0, 27410003.0}) 2
frozenset({10150032.0, 30380002.0, 27410003.0}) 2
frozenset({10150032.0, 27410003.0, 10000011.0}) 2
frozenset({10150032.0, 30380002.0, 27410003.0, 10000011.0}) 2
frozenset({24010380.0, 24101076.0}) 2
frozenset({14013009.0, 14072004.0}) 2
frozenset({25120036.0, 14831007.0}) 2
frozenset({14900007.0, 14831007.0}) 2
frozenset({25120036.0, 14900007.0, 14831007.0}) 2
frozenset({14082024.0, 30380003.0}) 2
frozenset({14260008.0, 15114015.0}) 2
frozenset({15114000.0, 14721035.0}) 2
frozenset({23110144.0, 15110007.0}) 2
frozenset({15110036.0, 15110007.0}) 2
frozenset({2311

frozenset({22000049.0, 25101044.0, 27000581.0, 27000582.0, 22030005.0}) 2
frozenset({27410000.0, 22000049.0, 25101044.0, 27000581.0, 27000582.0, 22030005.0}) 2
frozenset({22000049.0, 22102005.0}) 3
frozenset({22000049.0, 22102014.0}) 3
frozenset({22000049.0, 22102005.0, 22102014.0}) 2
frozenset({27410000.0, 22000049.0}) 3
frozenset({22000049.0, 25101044.0}) 3
frozenset({27410000.0, 22000049.0, 25101044.0}) 3
frozenset({27240000.0, 15000007.0}) 2
frozenset({27240000.0, 30380003.0, 15000007.0}) 2
frozenset({15200007.0, 15000007.0}) 2
frozenset({25120016.0, 15000007.0}) 2
frozenset({30380003.0, 15000007.0}) 3
frozenset({23110009.0, 15000007.0}) 3
frozenset({22671073.0, 27200924.0}) 2
frozenset({27410000.0, 27002558.0}) 2
frozenset({27410044.0, 27002558.0}) 2
frozenset({30380003.0, 27002558.0}) 4
frozenset({21801125.0, 10000006.0}) 2
frozenset({30380002.0, 21801125.0, 10000006.0}) 2
frozenset({30380003.0, 10000006.0}) 2
frozenset({27410000.0, 10000006.0}) 2
frozenset({30380002.0, 10000006.

frozenset({23110009.0, 30380002.0}) 9
frozenset({23110009.0, 27000582.0}) 3
frozenset({30380003.0, 27000582.0}) 4
frozenset({23110009.0, 30380003.0}) 11

 for pluno, threshold:  4
frozenset({27300274.0, 14082015.0}) 5
frozenset({14515001.0, 30380002.0}) 4
frozenset({14860025.0, 14860017.0}) 4
frozenset({14513033.0, 14258011.0}) 6
frozenset({27300274.0, 25100108.0}) 5
frozenset({27300274.0, 27300276.0}) 5
frozenset({15115003.0, 30380003.0}) 4
frozenset({30380003.0, 27002558.0}) 4
frozenset({22101001.0, 22102005.0}) 5
frozenset({27300274.0, 27002167.0}) 4
frozenset({27240000.0, 30380003.0}) 4
frozenset({22101000.0, 22102005.0}) 4
frozenset({22101000.0, 22102014.0}) 5
frozenset({30380003.0, 21801125.0}) 4
frozenset({22103001.0, 22100010.0}) 4
frozenset({22102000.0, 22100010.0}) 4
frozenset({25111048.0, 25111045.0}) 5
frozenset({27300273.0, 27410005.0}) 5
frozenset({27410000.0, 25101011.0}) 4
frozenset({30380003.0, 15130013.0}) 4
frozenset({22600000.0, 22102014.0}) 4
frozenset({22103011.0,

frozenset({30380002.0, 15503022.0}) 3
frozenset({14091021.0, 14082022.0}) 2
frozenset({30380002.0, 14091022.0}) 2
frozenset({30380003.0, 11533021.0}) 2
frozenset({30380003.0, 11500095.0}) 3
frozenset({30380002.0, 10310021.0}) 2
frozenset({10310051.0, 10310021.0}) 2
frozenset({25101002.0, 22501003.0}) 2
frozenset({22036000.0, 22501003.0}) 2
frozenset({22036000.0, 25101002.0, 22501003.0}) 2
frozenset({30380002.0, 10152012.0}) 2
frozenset({23110009.0, 27000037.0}) 2
frozenset({15119000.0, 14091006.0}) 2
frozenset({30380003.0, 24012151.0}) 2
frozenset({23110009.0, 24012151.0}) 2
frozenset({30380003.0, 11532039.0}) 2
frozenset({24102002.0, 24101002.0}) 2
frozenset({24102002.0, 23132063.0}) 2
frozenset({11533016.0, 30380003.0}) 2
frozenset({14000010.0, 30380003.0}) 2
frozenset({14000010.0, 14130055.0}) 2
frozenset({25121001.0, 22102005.0}) 2
frozenset({25121001.0, 30380002.0, 22102005.0}) 2
frozenset({25121001.0, 30380002.0}) 4
frozenset({23110009.0, 22021005.0}) 2
frozenset({30380002.0, 140

frozenset({22102000.0, 24010380.0}) 2
frozenset({22100010.0, 24010380.0}) 2
frozenset({24010380.0, 23132061.0}) 2
frozenset({24010380.0, 22102013.0}) 2
frozenset({15235001.0, 24010380.0}) 2
frozenset({22102009.0, 22020002.0}) 2
frozenset({22020002.0, 22102005.0}) 2
frozenset({22020002.0, 22111004.0}) 2
frozenset({22020002.0, 30380003.0}) 3
frozenset({30380003.0, 22001038.0}) 4
frozenset({23134003.0, 22007031.0}) 2
frozenset({23120001.0, 22007031.0}) 2
frozenset({23120001.0, 30380003.0, 22007031.0}) 2
frozenset({30380003.0, 22007031.0}) 5
frozenset({24101000.0, 23134002.0}) 2
frozenset({24101000.0, 22102005.0}) 2
frozenset({24101000.0, 22103005.0}) 2
frozenset({24101000.0, 22102014.0}) 2
frozenset({24101000.0, 30380002.0}) 2
frozenset({24101000.0, 22030011.0}) 2
frozenset({24101000.0, 30380003.0}) 3
frozenset({25130010.0, 30380003.0}) 2
frozenset({27410000.0, 25130010.0}) 3
frozenset({25130010.0, 25101044.0}) 5
frozenset({30380003.0, 10000006.0}) 2
frozenset({10141042.0, 10000006.0}) 2


frozenset({22100010.0, 22102005.0}) 7
frozenset({25120036.0, 27000581.0}) 3
frozenset({23110009.0, 25120036.0, 27000581.0}) 2
frozenset({30380003.0, 25120036.0, 27000581.0}) 2
frozenset({23110009.0, 30380003.0, 25120036.0, 27000581.0}) 2
frozenset({25120016.0, 25120036.0, 27000581.0}) 2
frozenset({30380002.0, 25120036.0}) 4
frozenset({25120016.0, 25120036.0}) 7
frozenset({25120016.0, 23110009.0, 25120036.0}) 3
frozenset({25120036.0, 25101044.0}) 7
frozenset({30380003.0, 25120036.0, 25101044.0}) 2
frozenset({25120016.0, 25120036.0, 25101044.0}) 3
frozenset({25120016.0, 23110009.0, 25120036.0, 25101044.0}) 2
frozenset({23110009.0, 25120036.0, 25101044.0}) 5
frozenset({30380003.0, 25120036.0}) 9
frozenset({23110009.0, 30380003.0, 25120036.0}) 2
frozenset({23110009.0, 25120036.0}) 9
frozenset({15119001.0, 23110001.0}) 2
frozenset({15119001.0, 23131002.0}) 2
frozenset({15119001.0, 27300274.0}) 2
frozenset({15119001.0, 27300274.0, 23110009.0}) 2
frozenset({27410000.0, 15119001.0}) 2
frozense

frozenset({23110009.0, 27300274.0}) 8
frozenset({27300274.0, 30380003.0}) 10
frozenset({25120016.0, 22102005.0}) 4
frozenset({25120016.0, 22036000.0}) 5
frozenset({25120016.0, 22102014.0}) 6
frozenset({25120016.0, 25101044.0}) 8
frozenset({25120016.0, 23110009.0}) 9
frozenset({25120016.0, 30380003.0}) 13
frozenset({25120016.0, 30380002.0}) 13
frozenset({25101044.0, 27000582.0}) 5
frozenset({25101044.0, 22102005.0}) 5
frozenset({27410000.0, 25101044.0}) 5
frozenset({25101044.0, 22102014.0}) 5
frozenset({30380002.0, 25101044.0}) 8
frozenset({30380003.0, 25101044.0}) 9
frozenset({23110009.0, 25101044.0}) 11
frozenset({23110001.0, 27000582.0}) 4
frozenset({27410000.0, 23110001.0}) 4
frozenset({23110001.0, 22102014.0}) 4
frozenset({23110001.0, 22102005.0}) 4
frozenset({23110001.0, 30380002.0}) 8
frozenset({23110001.0, 30380003.0}) 9
frozenset({23110001.0, 23110009.0}) 10
frozenset({22102014.0, 27000582.0}) 5
frozenset({23110009.0, 27000582.0}) 6
frozenset({30380002.0, 27000582.0}) 6
frozens